# 앙상블 및 스태킹 소개

이 캐글 노트북은 스태킹으로 알려진 앙상블 기법을 소개하고자 합니다.

앙상블 기법은 좀 덜 떨어지는 여러 대의 분류기가 학습 데이터를 학습 및 예측하여 나온 예측 결과를 다수결로 결정하는 방식과 동일한 분류기

를 사용하지만 각각의 학습 데이터를 샘플링을 달리하여 예측한 결과를 평균값으로 결정하는 방식이 있다.

스태킹기법은 각각의 분류기가 학습 후 예측한 결과를 볏짚 쌓듯이 차곡차곡 쌓아서 이를 다시 최종 메타 분류기가 다시 한번 학습 및 예측하는 기법.

해당 노트북은 [Stacking Starter][1]: by Faron  , [arthurtok][2] : by arthurtok 에 의해 쓰여진 캐글 노트북을 참고했습니다.

또한 최근의 [Introduction to Ensembling/Stacking in Python][3] 을 바탕으로 작성된 캐글임을 밝힙니다. 

피처엔지니어링에 대한 좋은 캐글은 [Titanic Best Working Classfier][4] 이 글을 참고하셔도 좋습니다.

"책을 읽는다는 것은 단순히 활자를 읽는다는 것이 아니라 , 그 시대를 읽는것이다"

  [1]: https://www.kaggle.com/mmueller/allstate-claims-severity/stacking-starter/run/390867
  [2]: https://www.kaggle.com/arthurtok/titanic/simple-stacking-with-xgboost-0-808
  [3]: https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python
  [4]: https://www.kaggle.com/sinakhorami/titanic/titanic-best-working-classifier

# Load library

In [ ]:
import pandas as pd
import numpy as np
import sklearn 
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from xgboost import XGBClassifier

# plotly를 이용한 시각화
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings 
warnings.filterwarnings('ignore')

# target directory for submission file
target_dir = '/kaggle/working/'

# Feature Exploration, Engineering and Cleaning 

범주형 피처에 대해서 숫자형으로 인코딩, 피처 엔지니어링, EDA및 데이터 정제 진행

In [ ]:
%cd ../input/titanic

In [ ]:
def load_dataset():
    """
    데이터셋 로드
    """
    train = pd.read_csv("train.csv")
    test  = pd.read_csv("test.csv")
    
    PassengerId = test["PassengerId"]
    
    display(train.head(n=2), "\n", train.shape, test.shape)
    display(train.info())
    display(test.info())
    
    full_data = [train, test]
    
    return train, test, PassengerId, full_data

# 데이터셋로드 
train, test, PassengerId, full_data = load_dataset()

범주형 변수(categorical variables)에서 인사이트를 찾아 새로운 파생변수를 생성하는 것이 피처 엔지니어링이라고 할 수 있다.

**피처 엔지니어링**

피처 엔지니어링에 대해서 아래 케글의 노트북을 참고하시면 좋을 것 같습니다.

[Titanic Best Working Classfier][1] : by Sina

  [1]: https://www.kaggle.com/sinakhorami/titanic/titanic-best-working-classifier

In [ ]:
# Age피처를 5개의 범주형 데이터로 나눈 결과르 빈도수 확인
pd.cut(train['Age'], 5).value_counts()

In [ ]:
# Fare피처를 4개의 범주형 데이터로 나눈 결과를 빈도수 확인
pd.cut(train['Fare'], 4).value_counts()

In [ ]:
full_data = [train, test]

# Some features of my own that I have added in
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

In [ ]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

## 시각화 

**Pearson Correlation Heatmap**

피처간의 상관관계를 한번 보도록 하자

In [ ]:
cmaps = ['Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'crest', 'crest_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r']

In [ ]:
plt.figure(figsize=(14, 12))
plt.title("Person Correlation of Features", y = 1.05, size = 15)
sns.heatmap(train.astype(float).corr(), linewidth=0.1, vmax=1.0, square=True, cmap="summer_r", linecolor="white", annot=True)

**Takeaway from the Plots**

FamilySize와 가장 연관이 있는 피처는 Parch(Parents and Children)이다. 

히트맵을 보면, 강한 상관관계를 갖는 피처들이 그렇게 많이 보이지 않는다는 점이다. ML모델을 학습하기에는 좋은 데이터의 분포라고 할수있다.(중복되는 피처가 적고, 학습세트에 과적합되는 사례가 줄어듬)


**Pairplots**

Pairplots는 한 피처와 다른 피처와의 데이터 분포도를 확인하는 것

[PairPlot Document](https://seaborn.pydata.org/generated/seaborn.pairplot.html?highlight=pairplot#seaborn.pairplot)

In [ ]:
data  =train[[u'Survived', u'Pclass', u'Sex', u'Age', u'Parch', u'Embarked', u'FamilySize', u'Title']]
g = sns.pairplot(data = data, hue='Survived', palette='seismic', size = 1.5, diag_kind='kde', diag_kws=dict(shade=True), plot_kws=dict(s=10))
g.set(xticklabels=[])

# 앙상블 및 스태킹 모델

* 스태킹 앙상블 모델 

### Helpers via Python Classes

sklearnHelper클래스를 생성하여 학습 및 예측을 수행하는 메소드 생성

In [ ]:
version  = sklearn.__version__
versionChk = True

ntrain = train.shape[0]
ntest  = test.shape[0]
SEED = 0
NFOLDS = 5

display("ntrain : ", ntrain)  # 891
display("ntest : ", ntest)    # 418
if version.split('.')[1] == '20':  #  # 0.20 Use this API
    from sklearn.model_selection import KFold  
    kf = KFold(n_splits = NFOLDS, random_state= SEED)
else:
    from sklearn.cross_validation import KFold  # 0.18 will be removed
    kf = KFold(ntrain, n_folds = NFOLDS, random_state=SEED)
    versionChk = False

In [ ]:
class SklearnHelper(object):
    """
    사이킷런의 객체를 상속받고, 사이킷런에서 제공하는 메소드를 구현하여 학습, 예측, feature_importance구현
    """
    def __init__(self, clf, seed, params):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def get_name(self):
        return self.clf.__class__.__name__
    
    def train(self, X_train, y_train):
        self.clf.fit(X_train, y_train)
        
    def predict(self, X):
        """
        인자 - 테스트 데이터셋트
        """
        return self.clf.predict(X)
    
    def feature_importances(self, X, y):
        """
        ML모델의 feature importance_출력
        """
        model = self.clf
        model.fit(X, y)
        display(model.feature_importances_)
        return model.feature_importances_   

### Out-of-Fold Predictions

학습 세트에만 최적화되는 오버피팅을 방지하기 위한 OOF Predict구현

In [ ]:
def loop_get_oof(clfs, X_train, y_train, X_test):
    """
    Out Of Fold Prediction를 모델의 갯수만큼 반복 수행
    """
    
    result = dict()
    for clf in clfs:
        print(clf.get_name())
        oof_train, oof_test = get_oof(clf, X_train, y_train, X_test)
        result[clf] = (oof_train, oof_test)
    return result

In [ ]:
oof_train = np.zeros((891,))
display('oof_train  :', oof_train.shape)

oof_test  = np.zeros((481, ))
display('oof_test  :', oof_test.shape)

oof_test_skf = np.empty((5, 481))
display('oof_test_skf  :', oof_test_skf.shape, oof_test_skf)

In [ ]:
def get_oof(clf, X_train, y_train, X_test):
    """
    oof Prediction
    """
    oof_train = np.zeros((ntrain,))
    oof_test  = np.zeros((ntest, ))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    # scikitlearn버전 0.18과 0.20버전에서 지원하는 API가 처리방식이 다름
    for i, (train_index, test_index) in enumerate(kf.split(X_train, y_train) if versionChk else kf):
        x_tr = X_train[train_index]
        y_tr = y_train[train_index]
        X_te = X_train[test_index]
        
        clf.train(x_tr, y_tr)
        
        # 학습데이터에서 검증용 데이터로 예측한 결과를 최종 학습데이터로 사용
        oof_train[test_index] = clf.predict(X_te)
        # 테스트 데이터로 예측한 결과를 최종 검증된 테스트 데이터로 사용
        oof_test_skf[i,:] = clf.predict(X_test)
        
    oof_test[:] = oof_test_skf.mean(axis=0) # 최종 평균값을 Out Of Fold 테스트 데이터로 사용
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

# Generating our Base First-Level Models 

먼저 앙상블의 첫번째 단계로 5개의 ML모델을 대상으로 한다.

 1. Random Forest classifier
 2. Extra Trees classifier
 3. AdaBoost classifer
 4. Gradient Boosting classifer
 5. Support Vector Machine

**Parameters**

**n_jobs** : 학습시에 사용할 CPU코어 수.  -1의 의미는 모든 CPU코아를 다 사용

**n_estimators** : ML모델 분류기의 수 ( 디폴트는 10)

**max_depth** : 트리의 최대 깊이 수(크게 설정될수록 오버피팅이 위험이 존재함을 유념)

**verbose** : 학습이 진행될 동안 출력결과를 보고 싶을 경우 설정( 0 - 결과를 보이지 않음, 1- 결과보기)

In [ ]:
# hyper parameter for RandomForestClassifier
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    'warm_start': True,
    #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'verbose': 0
}

# hyper parameter for ExtraTreesClassifier
et_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features':'sqrt',
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}


# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [ ]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svm = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

**Creating NumPy arrays out of our train and test sets**

학습 및 테스트 세트로 부터 모델을 학습시킬 데이터세트 생성

In [ ]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train["Survived"].ravel()  # ravel()으로 pd.Series타입으로 1차원 ndarray타입으로 변환, 타겟값을 1차원배열로 ML모델에 학습시 이렇게 들어감
train = train.drop(['Survived'], axis=1, errors='ignore')

# ML모델을 학습시키기 위해서는 ndarray로 변환이 필요
X_train = train.values  #  DataFrame을 ndarray로 변환  - (891, 11)   
X_test  = test.values   #  DataFrame를 ndarray로 변환  - (481, 11)

**Output of the First level Predictions** 

학습 및 테스트 데이터를 5개의 ML모델을 학습시키고, OOF Prediction을 수행

In [ ]:
# Create our OOF train and test predictions. These base results will be used as new features
clfs = [rf, et, ada, gb, svm]
result = loop_get_oof(clfs, X_train, y_train, X_test)

In [ ]:
def get_oof_train_test(clf):
    """
    OOF결과 리턴 받은 결과값에서 필요로 하는 분류기의 최종 학습 및 테스트 데이터셋 
    결과 가져오기 
    """
    r0 = [v[0] for k,v in result.items() if k.clf.__class__.__name__==clf]
    r1 = [v[1] for k,v in result.items() if k.clf.__class__.__name__==clf]
    
    return r0, r1

In [ ]:
rf_oof_train, rf_oof_test = get_oof_train_test('RandomForestClassifier')
et_oof_train, et_oof_test = get_oof_train_test('ExtraTreesClassifier')
ada_oof_train, ada_oof_test = get_oof_train_test('AdaBoostClassifier')
gb_oof_train, gb_oof_test = get_oof_train_test('GradientBoostingClassifier')
svm_oof_train, svm_oof_test = get_oof_train_test('SVC')

**Feature importances generated from the different classifiers**

* feature_importances_를 통해서 피처의 중요도 보기

In [ ]:
rf_feature = rf.feature_importances(X_train, y_train)
et_feature = et.feature_importances(X_train, y_train)
ada_feature = ada.feature_importances(X_train, y_train)
gb_feature = gb.feature_importances(X_train, y_train)

In [ ]:
rf_feature = rf.clf.feature_importances_
et_feature = et.clf.feature_importances_
ada_feature = ada.clf.feature_importances_
gb_feature = gb.clf.feature_importances_

* 위의 결과값을 변수에 할당 - 위의 형태로 바로 클래스객체 바로 호출해봄.

In [ ]:
rf_feature, et_feature, ada_feature, gb_feature

* 피처 중요도를 DataFrame형태로 생성

In [ ]:
cols = train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_feature,
     'Extra Trees  feature importances': et_feature,
      'AdaBoost feature importances': ada_feature,
      'Gradient Boost feature importances': gb_feature
    })

In [ ]:
feature_dataframe

In [ ]:
feature_dataframe.info()

**Interactive feature importances via Plotly scatterplots**

* Potly를 이용해 Scatter그리기
* [plotly tutorial for Python](https://plotly.com/python/)

* Scatter Plot using Plotly

In [ ]:
cols = feature_dataframe.columns.values[1:].tolist()  # features컬럼은 상수로 고정시키고 그 다음부터만 가져옴

for col in cols:
    
    trace = go.Scatter(
    y = feature_dataframe[col].values,
    x = feature_dataframe["features"].values,
    mode="markers",
    marker=dict(
        sizemode="diameter",
        sizeref = 1,
        size = 25,
        #       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe[col].values,
        colorscale = "Portland",
        showscale=True
    ),
    text = feature_dataframe["features"].values    
    )
    data = [trace]
    layout = go.Layout(
        autosize = True,
        title = col,
        hovermode="closest",
        #     xaxis= dict(
        #         title= 'Pop',
        #         ticklen= 5,
        #         zeroline= False,
        #         gridwidth= 2,
        #     ),
        yaxis = dict(
            title="Feature Importances",
            ticklen = 5,
            gridwidth = 2
        ),
        showlegend = False    
    )

    fig = go.Figure(data = data, layout =layout)
    py.iplot(fig, filename='Scatter2010')

* Feature Importance값의 평균값을 새로운 피처로 생성

In [ ]:
feature_dataframe["mean"] = feature_dataframe.mean(axis=1)
feature_dataframe.head(n=3)

**Plotly Barplot of Average Feature Importances**

피처 중요도의 평균을 바차트로 표시

In [ ]:
y = feature_dataframe["mean"].values
x = feature_dataframe["features"].values

# data region
data = [
    go.Bar(
            x = x,
            y = y,
            marker=dict(
                color = feature_dataframe["mean"].values,
                colorscale="Portland",
                showscale=True,
                reversescale=False
            ),
            opacity=0.6
    )    
]

#그래프의 배치 및 눈금 
layout = go.Layout(
    autosize=True,
    title="BarPlots of Mean Feature Importances",
    hovermode="closest",
    xaxis = dict(
        title="Feature mean",
        ticklen= 5,
        zeroline = False,
        gridwidth= 2
    ),
    yaxis = dict(
        title= "Feature Imporances",
        ticklen= 5,
        gridwidth=  2
    ),
    showlegend = False
)
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='bar-direct-labels')

# Second-Level Predictions from the First-level Output

**First-level output as new features**

In [ ]:
base_predictions_train = pd.DataFrame( {'RandomForest': np.array(rf_oof_train).ravel(),
     'ExtraTrees': np.array(et_oof_train).ravel(),
     'AdaBoost': np.array(ada_oof_train).ravel(),
      'GradientBoost': np.array(gb_oof_train).ravel()
    })
base_predictions_train.head()

** Correlation Heatmap of the Second Level Training set**

In [ ]:
data = [
    go.Heatmap(
        z = base_predictions_train.astype(float).corr().values,
        x = base_predictions_train.columns.values,
        y = base_predictions_train.columns.values,
        colorscale='Viridis',
        showscale=True,
        reversescale=True
    )
]
py.iplot(data, filename='labelled-heatmap')

In [ ]:
type(rf_oof_train[0])

In [ ]:
# 리턴한 한 타입이 리스트안에 배열로 들어가있으니까...ndarray타입으로 해야 하니 인덱스 [0]을 취한 것.
X_train = np.concatenate((et_oof_train[0], rf_oof_train[0], ada_oof_train[0], gb_oof_train[0], svm_oof_train[0]), axis = 1)
X_test  = np.concatenate((et_oof_test[0], rf_oof_test[0], ada_oof_test[0], gb_oof_test[0], svm_oof_test[0]), axis= 1)

1차 base estimator에서 나온 학습 및 테스트 세트를 병합해 메타 머신 러닝 모델에서 사용

### XGBoost를 이용한 메타모델 학습 및 예측

XGBoost를 최종 메타모델 분류기로 사용 할 것이고 자세한 내용은 공식 사이트를 참조하기 바랍니다.

[official Documentation][1].

  [1]: https://xgboost.readthedocs.io/en/latest/

In [ ]:
gbm = XGBClassifier(n_estimators = 2000,
                   learning_rate = 0.02,
                   max_depth=4,
                   min_child_weight=2,
                   gamma=0.9,
                   subsample=0.8,
                   colsample_bytree=0.8,
                   objective='binary:logistic',
                   nthreads=-1,
                   scale_pos_weight=1)
gbm.fit(X_train, y_train)
predictions = gbm.predict(X_test)

XGBoost 대표적인 파라미터:

**max_depth** : 트리의 깊이 제어(related to the machine learning model overfitting)

**gamma** : 오버피팅을 위한 제어값

**eta** : learning_rate와 유사하다고 보면 됨

**최종 결과 제출 파일**

In [ ]:
StackingSubmission = pd.DataFrame(
    {
                "PassengerId": PassengerId,
                "Survived": predictions
    }
)
display(StackingSubmission.head(n=3))

In [ ]:
StackingSubmission.to_csv(target_dir+ "StackingSubmission.csv", index=False)

### 결론

스태킹 및 앙상블 기법에 대한 좋은 자료 참조 

website MLWave: [Kaggle Ensembling Guide][1]. 

[1]: http://mlwave.com/kaggle-ensembling-guide/